# Chess Neural Network

Building a neural network that learns to play chess based on my gameplay data.

## Goal
Train a move prediction model to create an AI that plays like me.




In [3]:
# Importing necessary libraries
import pandas as pd
import ast
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np


## Step 1: Load Chess Data

Load cleaned data from csv file


In [4]:
df = pd.read_csv("/Users/riteshbhandari/Documents/Dokumentit – Ritesh - MacBook Pro/GitHub/Chess-engine/bot/data-analysis/cleaned_data.csv")
df.head()

,game_id,moves,num_moves,first_move
0,123118274906,"[('white', 'e4', True), ('black', 'e6', False)...",90,e4
1,123118510404,"[('white', 'd4', False), ('black', 'c5', True)...",141,d4
2,123118790014,"[('white', 'e4', False), ('black', 'e5', True)...",46,e4
3,123158939328,"[('white', 'e4', False), ('black', 'd5', True)...",88,e4
4,123160166430,"[('white', 'e4', True), ('black', 'e5', False)...",110,e4


# Step 2: Pre-processing

2.1 Turning features to numerical values (moves.. etc) 

2.2 Encoding the features to be trained

In [5]:
# convert string representation of list to actual list
df["moves"] = df["moves"].apply(ast.literal_eval)

# saving all the moves to single list to be encoded 
every_move = []
for game in df["moves"]:
    for move in game:
        every_move.append(move[1])

every_move[:10]

['e4', 'e6', 'd4', 'Qh4', 'Nc3', 'f5', 'Nf3', 'Qe7', 'e5', 'Qb4']

In [6]:
# getting all the unique moves
unique_moves = set(every_move)  # just the unique moves
print("Number of different moves:", len(unique_moves))
print()

# give move a number
move_to_number = {}

# turning integer back to moves (for future use)
number_to_move= {}

for i, move in enumerate(unique_moves):
    move_to_number[move] = i
    number_to_move[i] = move

# turning all the numbers into integers
number_moves = []
for move in every_move:
    number_moves.append(move_to_number[move])

# first 10 moves
print("First 10 moves as numbers: ")
print(number_moves[:10])
print()

# first 10 original moves
print("First 10 original moves: ")
print(every_move[:10]) 

Number of different moves: 1927

First 10 moves as numbers: 
[1318, 923, 1852, 359, 703, 422, 1232, 776, 668, 1292]

First 10 original moves: 
['e4', 'e6', 'd4', 'Qh4', 'Nc3', 'f5', 'Nf3', 'Qe7', 'e5', 'Qb4']


In [7]:
# Create colors per game
colors_per_game = []

for game in df["moves"]:
    game_colors = []
    for move in game:
        if move[0] == "white":
            game_colors.append(1)
        else:  # black
            game_colors.append(0)
    colors_per_game.append(game_colors)

# Add to DataFrame
df["colors"] = colors_per_game

# Check first row
print(df[["moves", "colors"]].iloc[0])


moves     [(white, e4, True), (black, e6, False), (white...
colors    [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, ...
Name: 0, dtype: object


In [8]:
# turn white or black to integers ( 1  = (White), 0 = (Black) )
teoriat_moves_per_game = []

for game in df["moves"]:
    game_teoriat = []
    for move in game:
        if move[2] == True:
            game_teoriat.append(1)
        else:
            game_teoriat.append(0)
    teoriat_moves_per_game.append(game_teoriat)

# Add to DataFrame
df["teoriat_moves"] = teoriat_moves_per_game

# Check first row
print(df[["moves", "teoriat_moves"]].iloc[0])

moves            [(white, e4, True), (black, e6, False), (white...
teoriat_moves    [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, ...
Name: 0, dtype: object


In [9]:
#combine all the features into a single list of tuples seperated by games 
combined_features_per_game = []

for game_idx in range(len(df)):
    game_combined = []
    moves = df["moves"].iloc[game_idx]
    colors = df["colors"].iloc[game_idx]
    teoriat = df["teoriat_moves"].iloc[game_idx]
    
    for i in range(len(moves)):
        # (color, move_as_integer, your_move)
        game_combined.append((
            colors[i],
            move_to_number[moves[i][1]],  # convert move to integer
            teoriat[i]
        ))
    # Append the combined features for the game
    combined_features_per_game.append(game_combined)

# Add to DataFrame
df["game_data"] = combined_features_per_game

# Check first game data
print(combined_features_per_game[0])

[(1, 1318, 1), (0, 923, 0), (1, 1852, 1), (0, 359, 0), (1, 703, 1), (0, 422, 0), (1, 1232, 1), (0, 776, 0), (1, 668, 1), (0, 1292, 0), (1, 726, 1), (0, 1586, 0), (1, 279, 1), (0, 953, 0), (1, 755, 1), (0, 533, 0), (1, 1357, 1), (0, 1740, 0), (1, 1425, 1), (0, 223, 0), (1, 1900, 1), (0, 1509, 0), (1, 924, 1), (0, 257, 0), (1, 241, 1), (0, 1006, 0), (1, 492, 1), (0, 1225, 0), (1, 1322, 1), (0, 1302, 0), (1, 1900, 1), (0, 438, 0), (1, 318, 1), (0, 1577, 0), (1, 183, 1), (0, 1775, 0), (1, 854, 1), (0, 1327, 0), (1, 1557, 1), (0, 256, 0), (1, 1515, 1), (0, 1315, 0), (1, 1821, 1), (0, 1623, 0), (1, 183, 1), (0, 1070, 0), (1, 630, 1), (0, 602, 0), (1, 1866, 1), (0, 506, 0), (1, 159, 1), (0, 947, 0), (1, 1641, 1), (0, 1740, 0), (1, 468, 1), (0, 1791, 0), (1, 183, 1), (0, 1353, 0), (1, 1557, 1), (0, 984, 0), (1, 540, 1), (0, 539, 0), (1, 1366, 1), (0, 1754, 0), (1, 1063, 1), (0, 739, 0), (1, 47, 1), (0, 102, 0), (1, 540, 1), (0, 739, 0), (1, 47, 1), (0, 102, 0), (1, 1178, 1), (0, 809, 0), (1, 6

In [10]:
# lets drop the columns with unneccessary data 
df = df.drop(columns=["moves", "colors", "teoriat_moves", "first_move","num_moves"])
df.head()

,game_id,game_data
0,123118274906,"[(1, 1318, 1), (0, 923, 0), (1, 1852, 1), (0, ..."
1,123118510404,"[(1, 1852, 0), (0, 1415, 1), (1, 130, 0), (0, ..."
2,123118790014,"[(1, 1318, 0), (0, 668, 1), (1, 1232, 0), (0, ..."
3,123158939328,"[(1, 1318, 0), (0, 130, 1), (1, 668, 0), (0, 9..."
4,123160166430,"[(1, 1318, 1), (0, 668, 0), (1, 1294, 1), (0, ..."


### Step 2.2:  Embedded layer Encoding 


In [11]:

unique_moves = set()
for game in df["game_data"]:
    for (color, move, your_move) in game:
        unique_moves.add(move)

print(f"Number of unique moves: {len(unique_moves)}")
print(f"Move range: {min(unique_moves)} to {max(unique_moves)}")

Number of unique moves: 1927
Move range: 0 to 1926
